In [68]:
from collections import deque, defaultdict
from itertools import cycle

import matplotlib.pyplot as plt
%matplotlib inline

# Day 9 Marble Mania

We play marbles by starting with a circle of marbles, initially with just 1 marble (0) placed in it, then elves take turns placing a marblem in between the first and second marbles to the right of the current position.

The new marble becomes the current position. Every 23rd marble isn't placed and rather adds to the score the player placing it, plus they remove and add to their score the 7th marble to the left of the current position.

The game goes on for a specific number of marbles.

In [1]:
test_inp = """10 players; last marble is worth 1618 points: high score is 8317
13 players; last marble is worth 7999 points: high score is 146373
17 players; last marble is worth 1104 points: high score is 2764
21 players; last marble is worth 6111 points: high score is 54718
30 players; last marble is worth 5807 points: high score is 37305
""".splitlines()
test_inp

['10 players; last marble is worth 1618 points: high score is 8317',
 '13 players; last marble is worth 7999 points: high score is 146373',
 '17 players; last marble is worth 1104 points: high score is 2764',
 '21 players; last marble is worth 6111 points: high score is 54718',
 '30 players; last marble is worth 5807 points: high score is 37305']

I'm using deque instead of a list since deque is faster for this purpose - lists have O(n) performance for inserts, deques are O(1) for appends. So the way inserts work in this list is:

- we have a current position, at the start thats zero
- we look at the two marbles on the right of our current one (considering the list is circular) and place the new one in b/w them - so basically place the marble in the second postion to the right of the current.
- every 23rd marble isn't placed, rather it adds players score, and the 7th marble to the left is removed and added to the players score
- keep doing this until the last marble is placed

So first up, taking a look at how I'd figure out where to place the marbles - the initial marbles should look like:

```
[-] (0)
[1]  0 (1)
[2]  0 (2) 1 
[3]  0  2  1 (3)
[4]  0 (4) 2  1  3 
[5]  0  4  2 (5) 1  3 
[6]  0  4  2  5  1 (6) 3 
[7]  0  4  2  5  1  6  3 (7)
[8]  0 (8) 4  2  5  1  6  3  7 
[9]  0  8  4 (9) 2  5  1  6  3  7 
```

In [49]:
print(m)
m.rotate(-1)
print(m)

deque([1, 6, 3, 7, 0, 8, 4, 2, 5])
deque([6, 3, 7, 0, 8, 4, 2, 5, 1])


In [69]:
m = deque([0]) # our cirqular list of marbles
pos = 0        # tracking current position
for i in range(1,4):
    print(i, m,)
    m.rotate(-1)
    print(i, m)
    m.append(i)
    print("#####", i, m)

1 deque([0])
1 deque([0])
##### 1 deque([0, 1])
2 deque([0, 1])
2 deque([1, 0])
##### 2 deque([1, 0, 2])
3 deque([1, 0, 2])
3 deque([0, 2, 1])
##### 3 deque([0, 2, 1, 3])


This works by making the current position always the end of the queue, and we insert the new marble b/w the first and second by first rotating the queue counter-clockwise, thus bringing the 1st marble to the right of the current one to the end of the queue, then we append the new marble there, placing it in between the first and second one and leaving it at the end of the queue.

So we don't even need to track the current marbles position, as its always at the end! First I starting coding up a whole Marbles object which used a list and kept track of the current position, then figured out where to insert the new marble in, but that was slow, tedious and way too much code. Deque's with rotates are awesome.

Now putting the above into a func to solve this:

In [81]:
def play_marbles(num_players=10, num_marbles=1618):
    marbles = deque([0])
    scores = [0 for _ in range(num_players)]

    for marble in range(1, num_marbles+1):
        if marble % 23 == 0:  # scoring round
            marbles.rotate(7) # brings the 7th marble to the left to the front
            scores[marble % num_players] += marble + marbles.pop()
            marbles.rotate(-1)
        else:                 # insert marble round    
            marbles.rotate(-1)
            marbles.append(marble)
    
    return max(scores)

assert play_marbles() == 8317
assert play_marbles(13, 7999) == 146373
assert play_marbles(17, 1104) == 2764
assert play_marbles(21, 6111) == 54718
assert play_marbles(30, 5807) == 37305

play_marbles(426, 72058)

424112

# Part 2

Is just the same as above but longer, the idea being that the solution should be fast enough, and using deques over lists makes it fast enough:

In [66]:
%%time

play_marbles(426, 72058 * 100)

CPU times: user 1.48 s, sys: 84.4 ms, total: 1.57 s
Wall time: 1.57 s


3487352628